In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Loading Data & Analysis

In [6]:
df = pd.read_csv("ALL Prices 1990-2021 mar.csv")

In [7]:
df.columns

Index(['month', 'year', 'town', 'flat_type', 'lease_rem', 'resale_price'], dtype='object')

In [8]:
df[['month', 'town', 'flat_type', 'lease_rem', 'resale_price']]

month        town flat_type  lease_rem  resale_price
0       1990-01  ANG MO KIO    4 ROOM         85       86000.0
1       1990-01  ANG MO KIO    3 ROOM         85       59000.0
2       1990-01  ANG MO KIO    3 ROOM         85       47200.0
3       1990-01  ANG MO KIO    3 ROOM         85       47200.0
4       1990-01  ANG MO KIO    4 ROOM         86       80000.0
...         ...         ...       ...        ...           ...
840913  2021-03      YISHUN    3 ROOM         64      273000.0
840914  2021-03      YISHUN    3 ROOM         62      268000.0
840915  2021-03      YISHUN    2 ROOM         93      260000.0
840916  2021-03      YISHUN    2 ROOM         91      250000.0
840917  2021-03      YISHUN    2 ROOM         91      210000.0

[840918 rows x 5 columns]

# Preprocessing Data 

In [9]:
df['month'] = df.apply(lambda row: int(row.month[5:]), axis=1)

In [10]:
def check_quarter(month, year, min_year):
    if month <= 3:
        return 4*(year - min_year)
    elif month <= 6:
        return 1 + 4*(year - min_year)
    elif month <= 9:
        return 2 + 4*(year - min_year)
    return 3 + 4*(year - min_year)

In [11]:
def check_same(v1, v2):
    if v1 == v2:
        return 1
    return 0

In [12]:
#Assign numerical value to each quarter. Q1 - Jan-March , Q2 - April-June, Q3 - July-September, Q4- October-December
oldest_house = min(df['year'])

df['quarter'] = df.apply(lambda row: check_quarter(row.month, row.year, oldest_house), axis = 1)

In [13]:
print(sorted(set(df['town'])))

['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH', 'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG', 'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST', 'KALLANG/WHAMPOA', 'LIM CHU KANG', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL', 'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES', 'TOA PAYOH', 'WOODLANDS', 'YISHUN']


In [14]:
print(sorted(set(df['flat_type'])))

['1 ROOM', '2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', 'MULTI GENERATION']


In [15]:
#towns less Ang Mo Kio
towns = sorted(list(set(df['town'])))[1:]

#flat types less 1 ROOM 
flat_types = sorted(list(set(df['flat_type'])))[1:]

In [16]:
#Create binary variables for each house type. Assigning 1 for the type of the house and 0 for everything else
for x in flat_types:
    df[x] = df.apply(lambda row: check_same(row.flat_type, x), axis = 1)

In [17]:
for x in towns:
    df[x] = df.apply(lambda row: check_same(row.town, x), axis =1)    

In [18]:
relevant_fields = ['quarter', 'lease_rem', 'resale_price',] + flat_types + towns

relevant_data = df[relevant_fields]


In [19]:
relevant_data

quarter  lease_rem  resale_price  2 ROOM  3 ROOM  4 ROOM  5 ROOM  \
0             0         85       86000.0       0       0       1       0   
1             0         85       59000.0       0       1       0       0   
2             0         85       47200.0       0       1       0       0   
3             0         85       47200.0       0       1       0       0   
4             0         86       80000.0       0       0       1       0   
...         ...        ...           ...     ...     ...     ...     ...   
840913      124         64      273000.0       0       1       0       0   
840914      124         62      268000.0       0       1       0       0   
840915      124         93      260000.0       1       0       0       0   
840916      124         91      250000.0       1       0       0       0   
840917      124         91      210000.0       1       0       0       0   

        EXECUTIVE  MULTI GENERATION  BEDOK  ...  PASIR RIS  PUNGGOL  \
0               0                 0      0  ...          0        0   
1               0                 0      0  ...          0        0   
2               0                 0      0  ...          0        0   
3               0                 0      0  ...          0        0   
4               0                 0      0  ...          0        0   
...           ...               ...    ...  ...        ...      ...   
840913          0                 0      0  ...          0        0   
840914          0                 0      0  ...          0        0   
840915          0                 0      0  ...          0        0   
840916          0                 0      0  ...          0        0   
840917          0                 0      0  ...          0        0   

        QUEENSTOWN  SEMBAWANG  SENGKANG  SERANGOON  TAMPINES  TOA PAYOH  \
0                0          0         0          0         0          0   
1                0          0         0          0         0          0   
2                0          0         0          0         0          0   
3                0          0         0          0         0          0   
4                0          0         0          0         0          0   
...            ...        ...       ...        ...       ...        ...   
840913           0          0         0          0         0          0   
840914           0          0         0          0         0          0   
840915           0          0         0          0         0          0   
840916           0          0         0          0         0          0   
840917           0          0         0          0         0          0   

        WOODLANDS  YISHUN  
0               0       0  
1               0       0  
2               0       0  
3               0       0  
4               0       0  
...           ...     ...  
840913          0       1  
840914          0       1  
840915          0       1  
840916          0       1  
840917          0       1  

[840918 rows x 35 columns]

In [9]:
#Saving Data to file

relevant_data.to_csv('housing price data.csv', index = False)

# Linear Regression

In [21]:
#relevant_data = pd.read_csv('housing price data.csv') #read data from csv

variable_fields = [x for x in relevant_data.columns if x != 'resale_price']

variables = relevant_data[variable_fields]
price = relevant_data['resale_price']
X_train,X_test, y_train, y_test = train_test_split(variables, price, test_size = 0.2, random_state = 100)

In [22]:
#Train regression model with train data
reg = LinearRegression()
reg.fit(X_train, y_train)

#Predicting with regression model
y_test_pred = reg.predict(X_test)
y_train_pred = reg.predict(X_train)

In [23]:
# Check the Goodness of Fit (on Train Data)
print("Goodness of Fit of Model\n") 

print("Train Dataset")
print("Coefficient of Determination: (R^2)\t", reg.score(X_train, y_train))
print("Mean Squared Error (MSE): \t\t", mean_squared_error(y_train, y_train_pred))

# Check the Goodness of Fit (on Test Data)
print("\nTest Dataset")
print("Coefficient of Determination (R^2): \t", reg.score(X_test, y_test))
print("Mean Squared Error (MSE): \t\t", mean_squared_error(y_test, y_test_pred))

Goodness of Fit of Model

Train Dataset
Coefficient of Determination: (R^2)	 0.7843926596531872
Mean Squared Error (MSE): 		 4909930494.112846

Test Dataset
Coefficient of Determination (R^2): 	 0.7858487815743768
Mean Squared Error (MSE): 		 4899547755.487672


In [24]:
equation = dict()
for x in range(0, len(variable_fields)):
    equation[variable_fields[x]] = reg.coef_[x]

In [25]:
equation['intercept'] =  reg.intercept_

In [26]:
equation

{'quarter': 3302.3392749539767,
 'lease_rem': 3427.0556389000676,
 '2 ROOM': 34698.554553258924,
 '3 ROOM': 118735.6953231857,
 '4 ROOM': 207380.99288000792,
 '5 ROOM': 298849.80524946994,
 'EXECUTIVE': 397862.37765586196,
 'MULTI GENERATION': 448419.32623058144,
 'BEDOK': -15169.669397215192,
 'BISHAN': 46611.28574482282,
 'BUKIT BATOK': -48918.507136133885,
 'BUKIT MERAH': 49047.9808480071,
 'BUKIT PANJANG': -93454.07568584676,
 'BUKIT TIMAH': 76079.69272978233,
 'CENTRAL AREA': 64699.65829918884,
 'CHOA CHU KANG': -85247.17868836987,
 'CLEMENTI': 13289.229427281332,
 'GEYLANG': -2510.1787126347,
 'HOUGANG': -47451.40076055824,
 'JURONG EAST': -43122.55606100025,
 'JURONG WEST': -77371.85631760187,
 'KALLANG/WHAMPOA': 23816.650950353356,
 'LIM CHU KANG': -78212.42593814516,
 'MARINE PARADE': 77365.01067319675,
 'PASIR RIS': -40382.46223712446,
 'PUNGGOL': -98548.34535758472,
 'QUEENSTOWN': 45212.47075313685,
 'SEMBAWANG': -145241.7345603187,
 'SENGKANG': -107186.95746376536,
 'SERANG

In [116]:
#Calculate price of 3 room flat in WOODLANDS in Feb 2025 with 75 year lease remaining

quarter = check_quarter(2, 2025, 1990)

lease = 75

price=quarter*equation['quarter']+lease*equation['lease_rem']+equation['WOODLANDS']+equation['3 ROOM']+equation['intercept']

price

375291.88614457013